In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\beatrizinacio-ieg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py

AttributeError: _ARRAY_API not found

In [2]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Analista"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [15]:
# pseudonimizar nome
def pseudonimizar_nome(nome_completo):
    # percorre o tamanho do nome recebido (cada letra)
    for i in range(len(nome_completo)):
        # cria uma string com o texto "Analista" e o número da posição
        nome = str("Analista " + str(i))
        # retorna o nome (mas aqui ele para logo na primeira volta do loop)
        return nome_completo

#Faz com que o nome do analista seja impresso como "Analista 1", "Analista 2", etc
df_anonimizado["nome_completo"] = [
    f"Analista {i+1}" for i in range(len(df_anonimizado))
]
#Imprime a anonimização 
print("==== Dados anonimizados ====")
display(df_anonimizado[['nome_completo']])


==== Dados anonimizados ====


,nome_completo
0,Analista 1
1,Analista 2
2,Analista 3
3,Analista 4
4,Analista 5


In [4]:
import bcrypt

def criptografar(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))


df_anonimizado["senha"] = df_anonimizado["senha"].apply(criptografar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["senha"]])





==== Dados anonimizados ====


,senha
0,$2b$12$lEMJsmNhLLJwTDGsg8/8HuYZOo3Nm1lRYhGZW1d...
1,$2b$12$3eZFV/SqIVTeJyDbkvoameVeHSZhpwYbKw55S9b...
2,$2b$12$kELOpe84n8CdMaOGOrLcfO.4MFmmkBVhg7WWeFS...
3,$2b$12$It950JZwd/UAOvQtwsTJKuKsKf2qbD8.CdyXGAg...
4,$2b$12$KDFVpeeiu/BiapEFPPsx7uVBJPN8SfJ.bd8fnYf...
5,$2b$12$Xt0cEAeWDGnQl8gHgf1dpex9fKvnGHytlyMltsn...
6,$2b$12$8V5olHJW.MBXgELWmol2IeHbG6qrtvKhqCHj8Ie...
7,$2b$12$U.yaX9f1Hw93OOZiWAkJPOMf1zYxyEOyWnUs0w/...
8,$2b$12$gxrEMKVArTSOmjcFTuRxae44shZzr64cYmAiGWe...


In [18]:
# Pseudonimização da data de contratação
from IPython.display import display


def pseudonimizar_data(dt_contratacao):
   # Garante que é string para que possa remover espaços 
    data = str(dt_contratacao).strip()
    
    # Remove hífen e barra
    data = data.replace("-", "").replace("/", "")
    
    # Função para anonimizar 4 primeiros caracteres
    if len(data) > 2:
        return data[:2] + "****"
    else:
        return data + "****"


# Aplica e mostra a pseudonimização
df_anonimizado["dt_contratacao"] = df_anonimizado["dt_contratacao"].apply(pseudonimizar_data)

print("==== Dados anonimizados ====")
display(df_anonimizado[["dt_contratacao"]])


==== Dados anonimizados ====


,dt_contratacao
0,20****
1,20****
2,20****
3,20****
4,20****


In [11]:
from IPython.display import display

# Função que transforma qualquer string em '*'
def pseudonimizar(cargo):
    # Garante string, remove espaços e hífens só se quiser ignorar na contagem
    texto = str(cargo).strip().replace(" ", "").replace("-", "")
    return "*" * len(texto)

# Aplica pseudonimização usando map diretamente na coluna
df_anonimizado["cargo"] = df_anonimizado["cargo"].map(pseudonimizar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cargo"]])


==== Dados anonimizados ====


,cargo
0,****
1,****
2,****
3,****
4,****


In [5]:
# Anonimização analista 

def anonimizar_digitos(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["cpf"] = df_anonimizado["cpf"].map(anonimizar_digitos)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cpf"]])

==== Dados anonimizados ====


,cpf
0,***********
1,***********
2,***********
3,***********
4,***********


In [8]:
# Pseudonimização email
def pseudonimizar(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anonimizado["email"] = df_anonimizado["email"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["email"]])


==== Dados anonimizados ====


,email
0,****@eitruck.com
1,****@eitruck.com
2,****@eitruck.com
3,****@eitruck.com
4,****@eitruck.com


In [13]:
# Anonimizar id_unidade substituindo dígitos por '*'
def anonimizar_digitos(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["telefone"] = df_anonimizado["telefone"].map(anonimizar_digitos)

print("==== Dados anonimizados ====")
display(df_anonimizado[["telefone"]])


==== Dados anonimizados ====


,telefone
0,***********
1,***********
2,***********
3,***********
4,***********


In [20]:

def anonimizar_digitos(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["id_unidade"] = df_anonimizado["id_unidade"].map(anonimizar_digitos)

print("==== Dados anonimizados ====")
display(df_anonimizado[["id_unidade"]])


==== Dados anonimizados ====


,id_unidade
0,*
1,*
2,*
3,*
4,*


In [21]:
display(df_anonimizado)

,id,id_unidade,cpf,dt_contratacao,nome_completo,email,senha,cargo,telefone
0,1,*,***********,20****,Analista 1,****@eitruck.com,60f6c3fdd8f2efc9aada81b663a5134f4b338b6fb80ad7...,****,***********
1,2,*,***********,20****,Analista 2,****@eitruck.com,ef92b778bafe771e89245b89ecbc08a44a4e166c066599...,****,***********
2,3,*,***********,20****,Analista 3,****@eitruck.com,daaad6e5604e8e17bd9f108d91e26afe6281dac8fda009...,****,***********
3,4,*,***********,20****,Analista 4,****@eitruck.com,a277c3ba4d39d7a49cc6507e1818e0d6ebc3d8621ccc63...,****,***********
4,5,*,***********,20****,Analista 5,****@eitruck.com,240be518fabd2724ddb6f04eeb1da5967448d7e831c08c...,****,***********


In [22]:
dados_originais_df.to_csv('original_analista.csv')
df_anonimizado.to_csv('pseudo_analista.csv')

In [59]:
df_anonimizado.to_sql(name='analista', con=engine_saida, if_exists='replace', index=False)

5